In [1]:
import pandas as pd

In [2]:
reddit_df = pd.read_pickle("reddit_posts.pkl.xz")

FileNotFoundError: ignored

# Tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Embedding, LeakyReLU, GRU
from tensorflow.keras.models import Sequential


In [ ]:
num_words = 10000  # Size of vocabulary obtained when preprocessing text data
embeddings_size = 512

In [ ]:
# model.summary()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
!pip install category_encoders

     |████████████████████████████████| 81kB 2.2MB/s eta 0:00:011


In [5]:
from category_encoders import OneHotEncoder

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
ce = OneHotEncoder(use_cat_names=True)

In [ ]:
subreddits = ce.fit_transform(reddit_df['subreddit'])
reddit_df = pd.concat([reddit_df, subreddits], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    reddit_df.drop(["up_votes", 'subreddit'], axis=1), reddit_df["up_votes"]
    )

In [ ]:
X_train

,title,text,subreddit_r/AskReddit,subreddit_r/gaming,subreddit_r/aww,subreddit_r/Music,subreddit_r/science,subreddit_r/worldnews,subreddit_r/videos,subreddit_r/movies,subreddit_r/Showerthoughts,subreddit_r/IAmA,subreddit_r/EarthPorn,subreddit_r/askscience,subreddit_r/Jokes,subreddit_r/explainlikeimfive,subreddit_r/books,subreddit_r/LifeProTips,subreddit_r/blog,subreddit_r/DIY,subreddit_r/sports,subreddit_r/nottheonion,subreddit_r/space,subreddit_r/gadgets,subreddit_r/television,subreddit_r/GetMotivated,subreddit_r/photoshopbattles,subreddit_r/listentothis,subreddit_r/UpliftingNews,subreddit_r/tifu,subreddit_r/InternetIsBeautiful,subreddit_r/history,subreddit_r/philosophy,subreddit_r/Futurology,subreddit_r/OldSchoolCool,subreddit_r/WritingPrompts,subreddit_r/nosleep,subreddit_r/personalfinance,subreddit_r/creepy,subreddit_r/TwoXChromosomes,subreddit_r/technology,subreddit_r/Fitness,subreddit_r/wholesomememes,subreddit_r/politics,subreddit_r/interestingasfuck,subreddit_r/WTF,subreddit_r/bestof,subreddit_r/travel,subreddit_r/oddlysatisfying,subreddit_r/leagueoflegends,subreddit_r/me_irl,subreddit_r/lifehacks,subreddit_r/NatureIsFuckingLit,subreddit_r/pcmasterrace,subreddit_r/dankmemes,subreddit_r/Whatcouldgowrong,subreddit_r/Tinder,subreddit_r/relationship_advice,subreddit_r/Minecraft,subreddit_r/PS4,subreddit_r/nba,subreddit_r/woahdude,subreddit_r/FoodPorn,subreddit_r/photography,subreddit_r/Overwatch,subreddit_r/Unexpected,subreddit_r/dadjokes,subreddit_r/relationships,subreddit_r/boardgames,subreddit_r/instant_regret,subreddit_r/programming,subreddit_r/PublicFreakout,subreddit_r/pokemon,subreddit_r/pokemongo
6255,I'm dyslexic so reading has always been hard f...,I'm 35 years old and have struggled with dysle...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18012,Through the Ages nearly giving me an aneurysm.,I was playing the new Through the Ages IOS app...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8379,TIFU by peeing in a girls mouth thinking i was...,Me and this girl met up after ages of speaking...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16092,So today my five-year-old daughter made me pro...,"She was eating watermelon, and she wanted to k...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
7374,"LPT: When writing an email, leave the To-field...",This avoids sending an unfinished email by acc...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6314,So my date took me to a bookstore yesterday...,I hope this isn't off topic for this subreddit...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6905,LPT — Quickly find out whether a power outage ...,"Generally, no power = no router = no Wi-Fi = n...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7097,LPT: If you live far away from a young family ...,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
376,What song did you fall in love with not becaus...,,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
def tokenize_and_pad(train, test, col):
    tokenizer = Tokenizer(num_words=num_words)
    tokenizer.fit_on_texts(train[col])
    train_tokens = tokenizer.texts_to_sequences(train[col])
    train_len = max([len(x) for x in train_tokens])
    train_padded = pad_sequences(train_tokens, train_len, padding="post")
    test_tokens = tokenizer.texts_to_sequences(test[col])
    test_padded = pad_sequences(test_tokens, train_len, padding="post")
    return (train_padded, test_padded)

In [ ]:
X_train_title_padded, X_test_title_padded = tokenize_and_pad(X_train, X_test, 'title')

In [ ]:
X_train_text_padded, X_test_text_padded = tokenize_and_pad(X_train, X_test, 'text')

In [ ]:
title_input = tf.keras.Input(
    shape=(None,), name="title"
)  # Variable-length sequence of ints
body_input = tf.keras.Input(shape=(None,), name="body")  # Variable-length sequence of ints
sub_reddit_input = tf.keras.Input(shape=(len(set(subreddits))), name="subreddit")

# Embed each word in the title into a 64-dimensional vector
title_features = Embedding(num_words, embeddings_size)(title_input)
# Embed each word in the text into a 64-dimensional vector
body_features = Embedding(num_words, embeddings_size)(body_input)

# Reduce sequence of embedded words in the title into a single 128-dimensional vector
title_features = GRU(128, return_sequences=True)(title_features)
title_features = GRU(128, return_sequences=True)(title_features)
title_features = GRU(64)(title_features)
# Reduce sequence of embedded words in the body into a single 32-dimensional vector
body_features = GRU(125, return_sequences=True)(body_features)
body_features = GRU(64)(body_features)

# Merge all available features into a single large vector via concatenation
x = tf.keras.layers.concatenate([title_features, body_features, sub_reddit_input])

# Stick a logistic regression for priority prediction on top of the features
# x = Dense(1024)(x)
# x = LeakyReLU()(x)
# # x = Dropout(0.5)(x)
# x = Dense(1024)(x)
# x = LeakyReLU()(x)
# # x = Dropout(0.5)(x)
# x = Dense(1024)(x)
# x = LeakyReLU()(x)
# # x = Dropout(0.5)(x)
x = Dense(1028)(x)
x = LeakyReLU()(x)
# # x = Dropout(0.5)(x)
x = Dense(1028)(x)
x = LeakyReLU()(x)
x = Dropout(0.5)(x)
x = Dense(1024)(x)
x = LeakyReLU()(x)
upvote_predictor = Dense(1, name="upvotes")(x)

model = tf.keras.Model([title_input, body_input, sub_reddit_input],upvote_predictor )


optimizer = tf.keras.optimizers.Adam(epsilon=0.1)
# loss = tf.keras.losses.MeanSquaredError()
# loss = tf.keras.losses.MeanSquaredLogarithmicError()
loss = tf.keras.losses.Huber(10000)

model.compile (
    optimizer, loss, ["mae"]
)

In [ ]:
stopping = tf.keras.callbacks.EarlyStopping('val_mae', 1, 5)

In [ ]:
model.fit(
    {
        "title": X_train_title_padded, 
     "body": X_train_text_padded,
     "subreddit": X_train.drop(["text","title"], axis = 1)}, y_train,
    epochs=1000, validation_data=(
        {
            "title": X_test_title_padded, 
         "body": X_test_text_padded,
         "subreddit": X_test.drop(["text","title"], axis = 1)}
         , y_test),
    callbacks=[stopping]
)

Epoch 1/1000
436/436 [==============================] - 351s 806ms/step - loss: 81224608.0000 - mae: 11704.1045 - val_loss: 33662172.0000 - val_mae: 6026.3867
Epoch 2/1000
436/436 [==============================] - 350s 804ms/step - loss: 36300516.0000 - mae: 6452.1855 - val_loss: 32536794.0000 - val_mae: 5881.9609
Epoch 3/1000
436/436 [==============================] - 351s 805ms/step - loss: 35383592.0000 - mae: 6301.9146 - val_loss: 33973260.0000 - val_mae: 6403.3735
Epoch 4/1000
436/436 [==============================] - 350s 803ms/step - loss: 35049700.0000 - mae: 6274.6548 - val_loss: 34242232.0000 - val_mae: 6430.6387
Epoch 5/1000
436/436 [==============================] - 352s 807ms/step - loss: 34070032.0000 - mae: 6127.7695 - val_loss: 32378198.0000 - val_mae: 5968.7798
Epoch 6/1000
436/436 [==============================] - 351s 806ms/step - loss: 32319522.0000 - mae: 5891.8291 - val_loss: 35019108.0000 - val_mae: 6105.5854
Epoch 7/1000
436/436 [=============================

In [ ]:
model.evaluate({"title": X_test_title_padded, "body": X_test_text_padded}, y_test)

In [ ]:
X_test_text_padded

array([[   0,    0,    0, ...,    0,    0,    0],
       [  26,   79,    1, ...,    0,    0,    0],
       [1658, 1210, 2420, ...,    0,    0,    0],
       ...,
       [  26,  737,   19, ...,    0,    0,    0],
       [   2,  105,  818, ...,    0,    0,    0],
       [  26,   71, 4019, ...,    0,    0,    0]], dtype=int32)

In [ ]:
X_test_title_padded

array([[  21,   54,  170, ...,    0,    0,    0],
       [   8,  737,  141, ...,    0,    0,    0],
       [ 937, 4264, 1437, ...,    0,    0,    0],
       ...,
       [  84,    5,    6, ...,    0,    0,    0],
       [  24,  231,    1, ...,    0,    0,    0],
       [  42,   22, 3116, ...,    0,    0,    0]], dtype=int32)

In [ ]:
model.evaluate()

# Sklearn with Basilica embeddings

In [ ]:
!pip install basilica

  Created wheel for basilica: filename=basilica-0.2.8-cp36-none-any.whl size=4711 sha256=ff6baafd57ca885f13cf73d4dbc33e5370c7d1941c2171ba38a814394db1024b
  Stored in directory: /root/.cache/pip/wheels/31/18/9f/46f6face8baf98e31b52bf91a0d76930ec76860f9e9211104d
Successfully built basilica


In [ ]:
import basilica

In [ ]:
def series_iterator(s:pd.Series):
    for i, v in s.iteritems():
        yield v

In [ ]:
def embed_sentences(sentences:pd.Series):
    with basilica.Connection('370a60d1-2938-b1bf-d813-0cb6954f5a0e') as c:
        embeddings = c.embed_sentences(
            series_iterator(sentences),
             model='reddit',
              batch_size = 200,
              timeout=120
              )
    return embeddings

In [ ]:
titles = reddit_df['title']

In [ ]:
titles

0        People who haven't pooped in 2019 yet, why are...
1        Would you watch a show where a billionaire CEO...
2        How would you feel about a feature where if so...
3                 Stan Lee has passed away at 95 years old
4        Reddit, how would you feel about a law that ba...
                               ...                        
18572                       Pokemon Go Update [2016-08-08]
18573    It boggles my mind that there is no way to sig...
18574    Is it really a coincidence that both Niantic a...
18575    The three footstep glitch has turned the game ...
18576    I feel like transfering second and third stage...
Name: title, Length: 18577, dtype: object

In [ ]:
title_embeddings = embed_sentences(titles)

In [ ]:
title_embeddings

<generator object Connection.embed at 0x7f587c70c9e8>

In [ ]:
reddit_df['title_embeddings'] = pd.Series(reddit_df['title_embeddings'])

KeyError: ignored

In [ ]:
reddit_df['title_embeddings'].head()

0    [0.182144, -0.0526975, 0.296982, -0.0468543, 0...
1    [0.0903963, -0.0630344, 0.183588, -0.132425, 0...
2    [0.20443, -0.299815, 0.278582, -0.330959, 0.28...
3    [0.0740549, -0.99451, 0.0833148, 0.17379, 0.43...
4    [0.254428, -0.0114105, 0.334044, -0.152447, 0....
Name: title_embeddings, dtype: object

In [ ]:
text = reddit_df['text']
text_embeddings = embed_sentences(text)
reddit_df['text_embeddings'] = pd.Series(text_embeddings)
reddit_df['text_embeddings'].head()

0    [-0.239577, 0.182857, 0.284122, -0.0861422, 0....
1    [-0.239577, 0.182857, 0.284122, -0.0861422, 0....
2    [-0.239577, 0.182857, 0.284122, -0.0861422, 0....
3    [0.0186579, -1.24461, 0.382696, 0.328033, 0.07...
4    [-0.239577, 0.182857, 0.284122, -0.0861422, 0....
Name: text_embeddings, dtype: object

In [ ]:
reddit_df.columns

Index(['text', 'up_votes', 'subreddit', 'title_embeddings', 'text_embeddings'], dtype='object')

In [ ]:
df.to_parquet("embeddings.parquet")

In [ ]:
df['title_embeddings'] = pd.Series(df['title_embeddings'])

In [ ]:
df['text_embeddings'] = pd.Series(df['text_embeddings'])

In [ ]:
df.head()

,up_votes,subreddit,title_embeddings,text_embeddings
0,221854,r/AskReddit,"[0.182144, -0.0526975, 0.296982, -0.0468543, 0...","[-0.239577, 0.182857, 0.284122, -0.0861422, 0...."
1,197524,r/AskReddit,"[0.0903963, -0.0630344, 0.183588, -0.132425, 0...","[-0.239577, 0.182857, 0.284122, -0.0861422, 0...."
2,186368,r/AskReddit,"[0.20443, -0.299815, 0.278582, -0.330959, 0.28...","[-0.239577, 0.182857, 0.284122, -0.0861422, 0...."
3,175339,r/AskReddit,"[0.0740549, -0.99451, 0.0833148, 0.17379, 0.43...","[0.0186579, -1.24461, 0.382696, 0.328033, 0.07..."
4,160311,r/AskReddit,"[0.254428, -0.0114105, 0.334044, -0.152447, 0....","[-0.239577, 0.182857, 0.284122, -0.0861422, 0...."


In [ ]:
import pandas as pd

df = pd.read_pickle("embeddings2.pkl.xz")

In [9]:
from sklearn.model_selection import train_test_split

In [ ]:
df = expanded

In [ ]:
X = df.drop(["up_votes", "subreddit"], axis=1)
y = df['up_votes']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from sklearn.linear_model import SGDRegressor as linear_model

In [ ]:
from xgboost import XGBRegressor as linear_model

In [ ]:
from sklearn.ensemble import RandomForestRegressor as linear_model

In [ ]:
model = linear_model()

In [ ]:
model.fit(X_train, y_train)

[05:42:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:

model.score(X_test, y_test)

0.4518445083917112

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_pred = model.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)

289856989.2280188

In [ ]:
mean_absolute_error(y_test, y_pred)

11289.522104992164

In [ ]:
zero = df['text_embeddings'].apply(lambda x: list(x))[0]

In [ ]:
type(zero)

list

In [ ]:
title_vector = df['title_embeddings'].apply(pd.Series)
title_vector.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.182144,-0.052698,0.296982,-0.046854,0.066862,-0.110474,0.109145,0.115977,-0.300374,0.012484,0.142182,-0.068124,-0.208033,0.149216,0.046058,0.014590,0.069055,0.108952,0.146693,-0.115352,0.096257,-0.078912,-0.141155,-0.117024,0.098890,-0.202187,0.057147,0.059443,0.018391,0.045539,0.011138,-0.157200,-0.041015,0.011421,-0.031243,0.112957,-0.087513,-0.060702,0.018499,-0.012571,...,-0.079505,0.104720,-0.026521,-0.118880,-0.133549,0.286074,0.133212,0.056164,0.017515,0.037554,-0.028962,0.015284,-0.267593,0.120774,-0.010770,0.151480,0.093855,0.009536,-0.033602,-0.096230,0.050461,0.054975,-0.051313,0.039797,-9.00595,0.057208,-0.009514,0.064110,-0.130201,0.078111,-0.004505,-0.211739,-0.026336,0.000743,0.333825,-0.151763,-0.064953,0.050184,-0.023745,-0.108041
1,0.090396,-0.063034,0.183588,-0.132425,0.218097,-0.061284,0.194140,0.099648,-0.110126,-0.059502,0.033862,0.062735,-0.222219,0.129082,-0.075026,0.102772,0.047288,0.282739,0.116910,-0.196864,-0.068466,-0.230079,0.000005,-0.075219,0.086746,-0.081459,0.198271,0.131429,0.164833,0.104517,0.061296,-0.225312,0.071521,-0.182611,-0.167435,0.170808,-0.015743,-0.016329,0.002535,0.039329,...,-0.027212,0.175481,-0.031569,-0.048201,-0.122848,0.264615,0.055259,-0.122896,-0.073748,0.025588,-0.026207,0.110434,-0.153051,0.239316,0.075326,0.244874,0.320699,0.046222,-0.022302,-0.087042,-0.110124,0.065037,-0.062415,0.060719,-8.97873,0.044415,0.048184,-0.005165,-0.039495,0.020332,-0.104020,-0.159283,0.081489,-0.094288,0.081652,-0.091267,0.003904,-0.030987,0.065567,0.015198
2,0.204430,-0.299815,0.278582,-0.330959,0.283490,0.167636,-0.000396,0.186616,-0.080640,-0.002074,0.083603,0.029153,-0.071468,-0.101955,-0.088183,0.027302,-0.042051,0.134516,0.101671,-0.028426,0.077124,-0.186997,-0.313275,-0.250894,0.080097,-0.078265,0.297395,0.140284,-0.008929,0.023978,0.057998,-0.050854,0.088528,0.067004,-0.133049,0.062508,-0.102993,-0.093009,-0.085430,0.076165,...,0.013532,0.145643,0.047052,-0.193214,-0.099032,0.441819,-0.026864,0.064393,-0.085460,-0.063976,0.001207,0.048360,-0.295799,0.250361,0.007193,0.002607,0.367675,0.064131,-0.044522,-0.257064,-0.175361,0.061791,-0.198281,-0.037277,-9.16263,0.137411,0.065680,0.157470,-0.289391,0.087019,-0.099552,-0.125894,0.119627,-0.012324,0.406567,-0.109556,-0.028680,0.201417,-0.064270,0.095350
3,0.074055,-0.994510,0.083315,0.173790,0.435514,0.489149,-0.442962,-0.515874,-0.247308,-0.210775,0.038379,0.199510,-0.283324,-0.483493,-0.898885,0.334158,0.353439,-1.088240,-1.086190,0.170191,-0.323830,-0.223867,-0.157821,-0.040704,-0.135643,0.109425,-0.167469,-0.357555,-0.247309,-0.306711,0.350606,-0.533339,0.937067,0.810881,-0.742399,-0.461994,-0.695902,-0.148049,-0.560312,-0.004402,...,-0.485970,-0.319889,0.264213,-0.837947,0.662875,0.054413,0.096752,-0.052930,-0.357925,0.054988,-0.023470,-0.015924,-0.816854,0.350448,0.005052,-0.324144,0.634460,-0.995354,-0.489603,0.468945,0.522310,1.203780,-0.918136,-0.362671,-1.22464,0.824677,0.418465,0.032358,-0.213618,-0.244904,0.089304,0.333229,0.727644,-0.946214,0.688935,-0.036943,-0.363213,0.905584,1.010070,0.210940
4,0.254428,-0.011411,0.334044,-0.152447,0.132704,-0.040200,0.080763,0.169764,-0.161985,-0.002933,0.075230,0.053451,-0.158602,0.013316,-0.063966,0.150923,0.022072,0.168311,0.124978,-0.229316,0.006148,-0.104214,-0.089166,-0.112851,0.157767,-0.022986,0.275391,0.152245,-0.022349,0.043627,0.068497,-0.072003,0.164927,0.128153,-0.097877,0.112834,0.025989,-0.035945,0.029443,0.111205,...,-0.048833,0.092955,0.000276,-0.139243,-0.096447,0.371564,0.056716,-0.018409,0.003533,-0.065163,0.013406,0.051614,-0.313355,0.178504,-0.017465,0.077003,0.257123,0.087590,-0.035367,-0.082249,-0.172318,0.057436,-0.017893,-0.038150,-9.09304,0.102770,0.073625,0.083781,-0.100582,0.020675,-0

In [ ]:
text_vector = df['text_embeddings'].apply(pd.Series)

In [ ]:
expanded = pd.concat((reddit_df[['subreddit', 'up_votes']], title_vector, text_vector), axis=1)

In [ ]:
reddit_df['up_votes']

0        221854
1        197524
2        186368
3        175339
4        160311
          ...  
18572      7189
18573      7168
18574      7117
18575      7108
18576      7097
Name: up_votes, Length: 18577, dtype: int64

In [ ]:
expanded.head()

,subreddit,up_votes,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,r/AskReddit,221854,0.182144,-0.052698,0.296982,-0.046854,0.066862,-0.110474,0.109145,0.115977,-0.300374,0.012484,0.142182,-0.068124,-0.208033,0.149216,0.046058,0.014590,0.069055,0.108952,0.146693,-0.115352,0.096257,-0.078912,-0.141155,-0.117024,0.098890,-0.202187,0.057147,0.059443,0.018391,0.045539,0.011138,-0.157200,-0.041015,0.011421,-0.031243,0.112957,-0.087513,-0.060702,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
1,r/AskReddit,197524,0.090396,-0.063034,0.183588,-0.132425,0.218097,-0.061284,0.194140,0.099648,-0.110126,-0.059502,0.033862,0.062735,-0.222219,0.129082,-0.075026,0.102772,0.047288,0.282739,0.116910,-0.196864,-0.068466,-0.230079,0.000005,-0.075219,0.086746,-0.081459,0.198271,0.131429,0.164833,0.104517,0.061296,-0.225312,0.071521,-0.182611,-0.167435,0.170808,-0.015743,-0.016329,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
2,r/AskReddit,186368,0.204430,-0.299815,0.278582,-0.330959,0.283490,0.167636,-0.000396,0.186616,-0.080640,-0.002074,0.083603,0.029153,-0.071468,-0.101955,-0.088183,0.027302,-0.042051,0.134516,0.101671,-0.028426,0.077124,-0.186997,-0.313275,-0.250894,0.080097,-0.078265,0.297395,0.140284,-0.008929,0.023978,0.057998,-0.050854,0.088528,0.067004,-0.133049,0.062508,-0.102993,-0.093009,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
3,r/AskReddit,175339,0.074055,-0.994510,0.083315,0.173790,0.435514,0.489149,-0.442962,-0.515874,-0.247308,-0.210775,0.038379,0.199510,-0.283324,-0.483493,-0.898885,0.334158,0.353439,-1.088240,-1.086190,0.170191,-0.323830,-0.223867,-0.157821,-0.040704,-0.135643,0.109425,-0.167469,-0.357555,-0.247309,-0.306711,0.350606,-0.533339,0.937067,0.810881,-0.742399,-0.461994,-0.695902,-0.148049,...,-0.123386,0.015488,0.403286,-0.721330,0.482571,-0.146677,0.164433,-0.812643,-0.319916,-0.050146,-0.242649,-0.401917,-1.172610,0.069800,0.322633,-0.226635,0.889395,-0.686808,-0.287803,-0.251987,0.591564,1.50253,-0.575015,-0.410068,-3.85364,0.476023,-0.121097,0.007005,-0.425543,-0.352296,-0.120428,-0.059130,0.952337,-1.308660,0.952924,0.261201,-0.729069,0.909033,0.791063,0.122177
4,r/AskReddit,160311,0.254428,-0.011411,0.334044,-0.152447,0.132704,-0.040200,0.080763,0.169764,-0.161985,-0.002933,0.075230,0.053451,-0.158602,0.013316,-0.063966,0.150923,0.022072,0.168311,0.124978,-0.229316,0.006148,-0.104214,-0.089166,-0.112851,0.157767,-0.022986,0.275391,0.152245,-0.022349,0.043627,0.068497,-0.072003,0.164927,0.128153,-0.097877,0.112834,0.025989,-0.035945,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.

In [ ]:
title_vector.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.182144,-0.052698,0.296982,-0.046854,0.066862,-0.110474,0.109145,0.115977,-0.300374,0.012484,0.142182,-0.068124,-0.208033,0.149216,0.046058,0.014590,0.069055,0.108952,0.146693,-0.115352,0.096257,-0.078912,-0.141155,-0.117024,0.098890,-0.202187,0.057147,0.059443,0.018391,0.045539,0.011138,-0.157200,-0.041015,0.011421,-0.031243,0.112957,-0.087513,-0.060702,0.018499,-0.012571,...,-0.079505,0.104720,-0.026521,-0.118880,-0.133549,0.286074,0.133212,0.056164,0.017515,0.037554,-0.028962,0.015284,-0.267593,0.120774,-0.010770,0.151480,0.093855,0.009536,-0.033602,-0.096230,0.050461,0.054975,-0.051313,0.039797,-9.00595,0.057208,-0.009514,0.064110,-0.130201,0.078111,-0.004505,-0.211739,-0.026336,0.000743,0.333825,-0.151763,-0.064953,0.050184,-0.023745,-0.108041
1,0.090396,-0.063034,0.183588,-0.132425,0.218097,-0.061284,0.194140,0.099648,-0.110126,-0.059502,0.033862,0.062735,-0.222219,0.129082,-0.075026,0.102772,0.047288,0.282739,0.116910,-0.196864,-0.068466,-0.230079,0.000005,-0.075219,0.086746,-0.081459,0.198271,0.131429,0.164833,0.104517,0.061296,-0.225312,0.071521,-0.182611,-0.167435,0.170808,-0.015743,-0.016329,0.002535,0.039329,...,-0.027212,0.175481,-0.031569,-0.048201,-0.122848,0.264615,0.055259,-0.122896,-0.073748,0.025588,-0.026207,0.110434,-0.153051,0.239316,0.075326,0.244874,0.320699,0.046222,-0.022302,-0.087042,-0.110124,0.065037,-0.062415,0.060719,-8.97873,0.044415,0.048184,-0.005165,-0.039495,0.020332,-0.104020,-0.159283,0.081489,-0.094288,0.081652,-0.091267,0.003904,-0.030987,0.065567,0.015198
2,0.204430,-0.299815,0.278582,-0.330959,0.283490,0.167636,-0.000396,0.186616,-0.080640,-0.002074,0.083603,0.029153,-0.071468,-0.101955,-0.088183,0.027302,-0.042051,0.134516,0.101671,-0.028426,0.077124,-0.186997,-0.313275,-0.250894,0.080097,-0.078265,0.297395,0.140284,-0.008929,0.023978,0.057998,-0.050854,0.088528,0.067004,-0.133049,0.062508,-0.102993,-0.093009,-0.085430,0.076165,...,0.013532,0.145643,0.047052,-0.193214,-0.099032,0.441819,-0.026864,0.064393,-0.085460,-0.063976,0.001207,0.048360,-0.295799,0.250361,0.007193,0.002607,0.367675,0.064131,-0.044522,-0.257064,-0.175361,0.061791,-0.198281,-0.037277,-9.16263,0.137411,0.065680,0.157470,-0.289391,0.087019,-0.099552,-0.125894,0.119627,-0.012324,0.406567,-0.109556,-0.028680,0.201417,-0.064270,0.095350
3,0.074055,-0.994510,0.083315,0.173790,0.435514,0.489149,-0.442962,-0.515874,-0.247308,-0.210775,0.038379,0.199510,-0.283324,-0.483493,-0.898885,0.334158,0.353439,-1.088240,-1.086190,0.170191,-0.323830,-0.223867,-0.157821,-0.040704,-0.135643,0.109425,-0.167469,-0.357555,-0.247309,-0.306711,0.350606,-0.533339,0.937067,0.810881,-0.742399,-0.461994,-0.695902,-0.148049,-0.560312,-0.004402,...,-0.485970,-0.319889,0.264213,-0.837947,0.662875,0.054413,0.096752,-0.052930,-0.357925,0.054988,-0.023470,-0.015924,-0.816854,0.350448,0.005052,-0.324144,0.634460,-0.995354,-0.489603,0.468945,0.522310,1.203780,-0.918136,-0.362671,-1.22464,0.824677,0.418465,0.032358,-0.213618,-0.244904,0.089304,0.333229,0.727644,-0.946214,0.688935,-0.036943,-0.363213,0.905584,1.010070,0.210940
4,0.254428,-0.011411,0.334044,-0.152447,0.132704,-0.040200,0.080763,0.169764,-0.161985,-0.002933,0.075230,0.053451,-0.158602,0.013316,-0.063966,0.150923,0.022072,0.168311,0.124978,-0.229316,0.006148,-0.104214,-0.089166,-0.112851,0.157767,-0.022986,0.275391,0.152245,-0.022349,0.043627,0.068497,-0.072003,0.164927,0.128153,-0.097877,0.112834,0.025989,-0.035945,0.029443,0.111205,...,-0.048833,0.092955,0.000276,-0.139243,-0.096447,0.371564,0.056716,-0.018409,0.003533,-0.065163,0.013406,0.051614,-0.313355,0.178504,-0.017465,0.077003,0.257123,0.087590,-0.035367,-0.082249,-0.172318,0.057436,-0.017893,-0.038150,-9.09304,0.102770,0.073625,0.083781,-0.100582,0.020675,-0

In [ ]:
text_vector.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.239577,0.182857,0.284122,-0.086142,0.207106,0.084421,0.253997,0.310796,-0.271986,-0.033349,-0.029973,0.060996,0.011152,0.016794,0.073788,0.049742,-0.120587,0.277185,-0.121792,-0.306350,0.089629,-0.016299,-0.364684,-0.291002,0.096310,-0.194393,0.088355,0.117249,0.018630,0.301229,0.073010,-0.302190,0.160194,0.217949,-0.125618,-0.342787,-0.145994,0.040412,0.097783,-0.077835,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
1,-0.239577,0.182857,0.284122,-0.086142,0.207106,0.084421,0.253997,0.310796,-0.271986,-0.033349,-0.029973,0.060996,0.011152,0.016794,0.073788,0.049742,-0.120587,0.277185,-0.121792,-0.306350,0.089629,-0.016299,-0.364684,-0.291002,0.096310,-0.194393,0.088355,0.117249,0.018630,0.301229,0.073010,-0.302190,0.160194,0.217949,-0.125618,-0.342787,-0.145994,0.040412,0.097783,-0.077835,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
2,-0.239577,0.182857,0.284122,-0.086142,0.207106,0.084421,0.253997,0.310796,-0.271986,-0.033349,-0.029973,0.060996,0.011152,0.016794,0.073788,0.049742,-0.120587,0.277185,-0.121792,-0.306350,0.089629,-0.016299,-0.364684,-0.291002,0.096310,-0.194393,0.088355,0.117249,0.018630,0.301229,0.073010,-0.302190,0.160194,0.217949,-0.125618,-0.342787,-0.145994,0.040412,0.097783,-0.077835,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
3,0.018658,-1.244610,0.382696,0.328033,0.075736,0.147204,0.208668,-0.325944,0.059597,-0.496547,-0.492126,0.137977,-1.014930,0.142615,-1.325580,0.137420,-0.196991,-0.905221,-0.677996,-0.173905,-0.212492,-0.643445,-0.244356,0.041128,-0.385912,0.982765,-0.919323,-0.352857,-0.428382,-0.236438,0.109246,-0.151543,0.706613,0.750051,-1.196410,-0.400648,-0.963530,-0.347007,0.242928,0.705486,...,-0.123386,0.015488,0.403286,-0.721330,0.482571,-0.146677,0.164433,-0.812643,-0.319916,-0.050146,-0.242649,-0.401917,-1.172610,0.069800,0.322633,-0.226635,0.889395,-0.686808,-0.287803,-0.251987,0.591564,1.50253,-0.575015,-0.410068,-3.85364,0.476023,-0.121097,0.007005,-0.425543,-0.352296,-0.120428,-0.059130,0.952337,-1.308660,0.952924,0.261201,-0.729069,0.909033,0.791063,0.122177
4,-0.239577,0.182857,0.284122,-0.086142,0.207106,0.084421,0.253997,0.310796,-0.271986,-0.033349,-0.029973,0.060996,0.011152,0.016794,0.073788,0.049742,-0.120587,0.277185,-0.121792,-0.306350,0.089629,-0.016299,-0.364684,-0.291002,0.096310,-0.194393,0.088355,0.117249,0.018630,0.301229,0.073010,-0.302190,0.160194,0.217949,-0.125618,-0.342787,-0.145994,0.040412,0.097783,-0.077835,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0

In [ ]:
expanded = pd.concat([reddit_df[['subreddit', 'up_votes']], title_vector, text_vector], axis=1)

In [ ]:
expanded.to_feather("expanded.feather")

In [ ]:
expanded.to_parquet("expanded.parquet")

In [ ]:
expanded.columns = expanded.columns.astype(str)

In [ ]:
expanded.head()

,subreddit,up_votes,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,r/AskReddit,221854,0.182144,-0.052698,0.296982,-0.046854,0.066862,-0.110474,0.109145,0.115977,-0.300374,0.012484,0.142182,-0.068124,-0.208033,0.149216,0.046058,0.014590,0.069055,0.108952,0.146693,-0.115352,0.096257,-0.078912,-0.141155,-0.117024,0.098890,-0.202187,0.057147,0.059443,0.018391,0.045539,0.011138,-0.157200,-0.041015,0.011421,-0.031243,0.112957,-0.087513,-0.060702,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
1,r/AskReddit,197524,0.090396,-0.063034,0.183588,-0.132425,0.218097,-0.061284,0.194140,0.099648,-0.110126,-0.059502,0.033862,0.062735,-0.222219,0.129082,-0.075026,0.102772,0.047288,0.282739,0.116910,-0.196864,-0.068466,-0.230079,0.000005,-0.075219,0.086746,-0.081459,0.198271,0.131429,0.164833,0.104517,0.061296,-0.225312,0.071521,-0.182611,-0.167435,0.170808,-0.015743,-0.016329,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
2,r/AskReddit,186368,0.204430,-0.299815,0.278582,-0.330959,0.283490,0.167636,-0.000396,0.186616,-0.080640,-0.002074,0.083603,0.029153,-0.071468,-0.101955,-0.088183,0.027302,-0.042051,0.134516,0.101671,-0.028426,0.077124,-0.186997,-0.313275,-0.250894,0.080097,-0.078265,0.297395,0.140284,-0.008929,0.023978,0.057998,-0.050854,0.088528,0.067004,-0.133049,0.062508,-0.102993,-0.093009,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
3,r/AskReddit,175339,0.074055,-0.994510,0.083315,0.173790,0.435514,0.489149,-0.442962,-0.515874,-0.247308,-0.210775,0.038379,0.199510,-0.283324,-0.483493,-0.898885,0.334158,0.353439,-1.088240,-1.086190,0.170191,-0.323830,-0.223867,-0.157821,-0.040704,-0.135643,0.109425,-0.167469,-0.357555,-0.247309,-0.306711,0.350606,-0.533339,0.937067,0.810881,-0.742399,-0.461994,-0.695902,-0.148049,...,-0.123386,0.015488,0.403286,-0.721330,0.482571,-0.146677,0.164433,-0.812643,-0.319916,-0.050146,-0.242649,-0.401917,-1.172610,0.069800,0.322633,-0.226635,0.889395,-0.686808,-0.287803,-0.251987,0.591564,1.50253,-0.575015,-0.410068,-3.85364,0.476023,-0.121097,0.007005,-0.425543,-0.352296,-0.120428,-0.059130,0.952337,-1.308660,0.952924,0.261201,-0.729069,0.909033,0.791063,0.122177
4,r/AskReddit,160311,0.254428,-0.011411,0.334044,-0.152447,0.132704,-0.040200,0.080763,0.169764,-0.161985,-0.002933,0.075230,0.053451,-0.158602,0.013316,-0.063966,0.150923,0.022072,0.168311,0.124978,-0.229316,0.006148,-0.104214,-0.089166,-0.112851,0.157767,-0.022986,0.275391,0.152245,-0.022349,0.043627,0.068497,-0.072003,0.164927,0.128153,-0.097877,0.112834,0.025989,-0.035945,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.

In [ ]:
from tqdm.notebook import tqdm
def column_namer(df:pd.DataFrame):
    yield "subreddit"
    yield "up_votes"
    for i in tqdm(range(df.shape[1]-2)):
        yield str(i)

In [ ]:
expanded.columns = list(column_namer(expanded))

In [ ]:
expanded.head()

,subreddit,up_votes,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,r/AskReddit,221854,0.182144,-0.052698,0.296982,-0.046854,0.066862,-0.110474,0.109145,0.115977,-0.300374,0.012484,0.142182,-0.068124,-0.208033,0.149216,0.046058,0.014590,0.069055,0.108952,0.146693,-0.115352,0.096257,-0.078912,-0.141155,-0.117024,0.098890,-0.202187,0.057147,0.059443,0.018391,0.045539,0.011138,-0.157200,-0.041015,0.011421,-0.031243,0.112957,-0.087513,-0.060702,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
1,r/AskReddit,197524,0.090396,-0.063034,0.183588,-0.132425,0.218097,-0.061284,0.194140,0.099648,-0.110126,-0.059502,0.033862,0.062735,-0.222219,0.129082,-0.075026,0.102772,0.047288,0.282739,0.116910,-0.196864,-0.068466,-0.230079,0.000005,-0.075219,0.086746,-0.081459,0.198271,0.131429,0.164833,0.104517,0.061296,-0.225312,0.071521,-0.182611,-0.167435,0.170808,-0.015743,-0.016329,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
2,r/AskReddit,186368,0.204430,-0.299815,0.278582,-0.330959,0.283490,0.167636,-0.000396,0.186616,-0.080640,-0.002074,0.083603,0.029153,-0.071468,-0.101955,-0.088183,0.027302,-0.042051,0.134516,0.101671,-0.028426,0.077124,-0.186997,-0.313275,-0.250894,0.080097,-0.078265,0.297395,0.140284,-0.008929,0.023978,0.057998,-0.050854,0.088528,0.067004,-0.133049,0.062508,-0.102993,-0.093009,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
3,r/AskReddit,175339,0.074055,-0.994510,0.083315,0.173790,0.435514,0.489149,-0.442962,-0.515874,-0.247308,-0.210775,0.038379,0.199510,-0.283324,-0.483493,-0.898885,0.334158,0.353439,-1.088240,-1.086190,0.170191,-0.323830,-0.223867,-0.157821,-0.040704,-0.135643,0.109425,-0.167469,-0.357555,-0.247309,-0.306711,0.350606,-0.533339,0.937067,0.810881,-0.742399,-0.461994,-0.695902,-0.148049,...,-0.123386,0.015488,0.403286,-0.721330,0.482571,-0.146677,0.164433,-0.812643,-0.319916,-0.050146,-0.242649,-0.401917,-1.172610,0.069800,0.322633,-0.226635,0.889395,-0.686808,-0.287803,-0.251987,0.591564,1.50253,-0.575015,-0.410068,-3.85364,0.476023,-0.121097,0.007005,-0.425543,-0.352296,-0.120428,-0.059130,0.952337,-1.308660,0.952924,0.261201,-0.729069,0.909033,0.791063,0.122177
4,r/AskReddit,160311,0.254428,-0.011411,0.334044,-0.152447,0.132704,-0.040200,0.080763,0.169764,-0.161985,-0.002933,0.075230,0.053451,-0.158602,0.013316,-0.063966,0.150923,0.022072,0.168311,0.124978,-0.229316,0.006148,-0.104214,-0.089166,-0.112851,0.157767,-0.022986,0.275391,0.152245,-0.022349,0.043627,0.068497,-0.072003,0.164927,0.128153,-0.097877,0.112834,0.025989,-0.035945,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.24373,-0.124181,-0.023625,-8.46837,-0.15559

In [ ]:
# 

# Tensorflow with Basilia embeddings

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU

In [ ]:
import pandas as pd

In [ ]:
reddit_df = pd.read_feather("expanded.feather")

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    reddit_df.drop(["up_votes", 'subreddit'], axis=1), reddit_df["up_votes"]
    )

In [ ]:
model = Sequential([
                    Dense(1024),
                    LeakyReLU(),
                    Dense(512),
                    LeakyReLU(),
                    Dense(512),
                    LeakyReLU(),
                    Dense(512),
                    LeakyReLU(),
                    Dense(1)
])

In [ ]:
optimizer = tf.keras.optimizers.Adam(epsilon=0.1)
# loss = tf.keras.losses.MeanSquaredError()
# loss = tf.keras.losses.MeanSquaredLogarithmicError()
loss = tf.keras.losses.Huber(1000)

model.compile (
    optimizer, loss, ["mse, ""mae"]
)

In [ ]:
stopping = tf.keras.callbacks.EarlyStopping('val_mae', 1, 25)

In [ ]:
model.fit(
    X_train, y_train,
    epochs=1000,
     validation_data=(X_test, y_test),
    callbacks=[stopping]
)

Epoch 1/1000
436/436 [==============================] - 12s 29ms/step - loss: 4917489.0000 - mae: 5384.8618 - val_loss: 8637050.0000 - val_mae: 9108.1162
Epoch 2/1000
436/436 [==============================] - 12s 29ms/step - loss: 4745751.5000 - mae: 5210.4897 - val_loss: 8578394.0000 - val_mae: 9051.4062
Epoch 3/1000
436/436 [==============================] - 12s 28ms/step - loss: 4814850.5000 - mae: 5280.2622 - val_loss: 8976645.0000 - val_mae: 9452.4580
Epoch 4/1000
436/436 [==============================] - 12s 28ms/step - loss: 4831089.5000 - mae: 5295.3223 - val_loss: 8388234.0000 - val_mae: 8865.7646
Epoch 5/1000
436/436 [==============================] - 12s 28ms/step - loss: 4648857.5000 - mae: 5114.4092 - val_loss: 8277718.0000 - val_mae: 8744.5830
Epoch 6/1000
436/436 [==============================] - 12s 28ms/step - loss: 4535278.0000 - mae: 4997.2607 - val_loss: 8504876.0000 - val_mae: 8974.3438
Epoch 7/1000
436/436 [==============================] - 13s 30ms/step - loss

# Autoencoder to reduce dim

In [ ]:
input = tf.keras.layers.Input(shape=(1536))
encoder = Dense(1024, "relu")(input)
encoder = Dense(512, "relu")(encoder)
encoder = Dense(256, "relu")(encoder)

decoder = Dense(512, "relu")(encoder)
decoder = Dense(1024, "relu")(decoder)
decoder = Dense(1536)(decoder)
output = decoder

In [ ]:
model = tf.keras.Model(input, output)

In [ ]:
model.compile("adam", "mse",["acc"])

In [ ]:
model.fit(X_train, X_train, epochs = 10)

Epoch 1/10
436/436 [==============================] - 23s 52ms/step - loss: 0.0235 - acc: 0.8150
Epoch 2/10
436/436 [==============================] - 22s 52ms/step - loss: 0.0142 - acc: 0.8599
Epoch 3/10
436/436 [==============================] - 22s 51ms/step - loss: 0.0125 - acc: 0.8673
Epoch 4/10
436/436 [==============================] - 23s 53ms/step - loss: 0.0115 - acc: 0.8732
Epoch 5/10
436/436 [==============================] - 22s 51ms/step - loss: 0.0108 - acc: 0.8799
Epoch 6/10
436/436 [==============================] - 22s 51ms/step - loss: 0.0103 - acc: 0.8806
Epoch 7/10
436/436 [==============================] - 22s 52ms/step - loss: 0.0101 - acc: 0.8791
Epoch 8/10
436/436 [==============================] - 22s 51ms/step - loss: 0.0097 - acc: 0.8840
Epoch 9/10
436/436 [==============================] - 22s 51ms/step - loss: 0.0095 - acc: 0.8844
Epoch 10/10
436/436 [==============================] - 22s 51ms/step - loss: 0.0091 - acc: 0.8868


In [ ]:
model.predict([X_train.iloc[:1]])[0][0:4]

array([0.00832764, 0.22634405, 0.35664377, 0.0007112 ], dtype=float32)

In [ ]:
[X_train.iloc[0].tolist(),][0][0:4]

[0.00601014, 0.227101, 0.431232, -0.0279421]

In [ ]:
type(encoder)

tensorflow.python.framework.ops.Tensor

In [ ]:
model.layers[3]

In [ ]:
autoencoder = model
for layer in autoencoder.layers:
    layer.trainable=False
model.summary()

Model: "functional_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 1536)]            0         
_________________________________________________________________
dense_134 (Dense)            (None, 1024)              1573888   
_________________________________________________________________
dense_135 (Dense)            (None, 512)               524800    
_________________________________________________________________
dense_136 (Dense)            (None, 256)               131328    
_________________________________________________________________
dense_137 (Dense)            (None, 512)               131584    
_________________________________________________________________
dense_138 (Dense)            (None, 1024)              525312    
_________________________________________________________________
dense_139 (Dense)            (None, 1536)            

# Autoencoder + NN

In [ ]:
predictor = Dense(256, "relu", kernel_regularizer=tf.keras.regularizers.l2(0.1), bias_regularizer=tf.keras.regularizers.l2(0.1))(encoder)
Predictor = Dropout(0.5)(predictor)
predictor = Dense(256, "relu", kernel_regularizer=tf.keras.regularizers.l2(0.1), bias_regularizer=tf.keras.regularizers.l2(0.1))(predictor)
Predictor = Dropout(0.5)(predictor)
predictor = Dense(256, "relu", kernel_regularizer=tf.keras.regularizers.l2(0.1), bias_regularizer=tf.keras.regularizers.l2(0.1))(predictor)
Predictor = Dropout(0.5)(predictor)
predictor = Dense(1)(predictor)
upvote_predictor = predictor


In [ ]:
predictor

<tf.Tensor 'dense_167/BiasAdd:0' shape=(None, 1) dtype=float32>

In [ ]:
model = tf.keras.Model(input,upvote_predictor )


optimizer = tf.keras.optimizers.Adam(epsilon=0.01)
# loss = tf.keras.losses.MeanSquaredError()
# loss = tf.keras.losses.MeanSquaredLogarithmicError()
loss = tf.keras.losses.Huber(10000)

model.compile (
    optimizer, loss, ["mse", "mae"]
)

In [ ]:
stopping = tf.keras.callbacks.EarlyStopping('val_mae', 1, 5)

In [ ]:
model.fit(
    X_train, y_train,
    epochs=1000, validation_data=(X_test, y_test),
    callbacks=[stopping]
)

Epoch 1/1000
436/436 [==============================] - 5s 11ms/step - loss: 118118152.0000 - mse: 605566144.0000 - mae: 15918.3799 - val_loss: 103745328.0000 - val_mse: 508790336.0000 - val_mae: 14379.0391
Epoch 2/1000
436/436 [==============================] - 4s 10ms/step - loss: 93462520.0000 - mse: 404163168.0000 - mae: 13312.4053 - val_loss: 90515608.0000 - val_mse: 381173664.0000 - val_mae: 13079.1602
Epoch 3/1000
436/436 [==============================] - 4s 10ms/step - loss: 80906344.0000 - mse: 336550112.0000 - mae: 11890.4121 - val_loss: 80464592.0000 - val_mse: 367192544.0000 - val_mae: 11748.9248
Epoch 4/1000
436/436 [==============================] - 5s 10ms/step - loss: 74687184.0000 - mse: 315335552.0000 - mae: 11152.9551 - val_loss: 80096128.0000 - val_mse: 342045440.0000 - val_mae: 11899.6855
Epoch 5/1000
436/436 [==============================] - 5s 10ms/step - loss: 71830968.0000 - mse: 301632000.0000 - mae: 10810.8975 - val_loss: 76391208.0000 - val_mse: 354808768.

# Autoendoer + sklearn models

In [ ]:
encoded = tf.keras.Model(input, encoder)

In [10]:
!pip install xgboost 

In [ ]:
from sklearn.linear_model import ElasticNetCV as sk_model

In [ ]:
from lightgbm import LGBMRegressor as sk_model

In [11]:
from xgboost import XGBRegressor as sk_model

In [ ]:
model = sk_model()

In [ ]:
model.fit(encoded.predict(X_train), y_train)

ValueError: ignored

In [ ]:
model.score(encoded.predict(X_test), y_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(y_test, model.predict(encoded.predict(X_test)))

ValueError: ignored

In [ ]:
!unxz "embeddings.pq(1).xz"

In [ ]:
df = pd.read_parquet("embeddings.pq(1)")

In [ ]:
df.head(1)

,up_votes,subreddit,title_embeddings,text_embeddings
0,221854,r/AskReddit,"[0.182144, -0.0526975, 0.296982, -0.0468543, 0...","[-0.239577, 0.182857, 0.284122, -0.0861422, 0...."


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(["up_votes", 'subreddit'], axis=1), df["up_votes"]
    )

In [ ]:
X_train.head()

,title_embeddings,text_embeddings
14398,"[0.0267503, -0.098055, 0.199862, -0.0685595, 0...","[-0.239577, 0.182857, 0.284122, -0.0861422, 0...."
7164,"[0.0193013, -0.0542681, 0.306836, -0.0460507, ...","[-0.239577, 0.182857, 0.284122, -0.0861422, 0...."
16879,"[-0.108175, -0.948139, 0.287051, -0.0746186, 0...","[-0.279692, -0.564432, 0.462589, 0.0895764, 0...."
15784,"[0.0553842, 0.109612, 0.324887, -0.265758, 0.2...","[0.109471, -0.191406, 0.534527, -0.285127, 0.1..."
3050,"[0.0601381, 0.102246, 0.318399, -0.00483384, 0...","[0.0452943, -0.234401, 0.0393467, -0.00753656,..."


In [ ]:
model.fit(X_train, y_train)

ValueError: ignored

In [ ]:
model.score(X_train, y_train)

In [5]:
reddit_df = pd.read_parquet("embeddings.parquet")

In [13]:
title_embeddings = reddit_df['title_embeddings'].apply(pd.Series)

In [13]:
title_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,0.182144,-0.052698,0.296982,-0.046854,0.066862,-0.110474,0.109145,0.115977,-0.300374,0.012484,0.142182,-0.068124,-0.208033,0.149216,0.046058,0.014590,0.069055,0.108952,0.146693,-0.115352,0.096257,-0.078912,-0.141155,-0.117024,0.098890,-0.202187,0.057147,0.059443,0.018391,0.045539,0.011138,-0.157200,-0.041015,0.011421,-0.031243,0.112957,-0.087513,-0.060702,0.018499,-0.012571,...,-0.079505,0.104720,-0.026521,-0.118880,-0.133549,0.286074,0.133212,0.056164,0.017515,0.037554,-0.028962,0.015284,-0.267593,0.120774,-0.010770,0.151480,0.093855,0.009536,-0.033602,-0.096230,0.050461,0.054975,-0.051313,0.039797,-9.00595,0.057208,-0.009514,0.064110,-0.130201,0.078111,-0.004505,-0.211739,-0.026336,0.000743,0.333825,-0.151763,-0.064953,0.050184,-0.023745,-0.108041
1,0.090396,-0.063034,0.183588,-0.132425,0.218097,-0.061284,0.194140,0.099648,-0.110126,-0.059502,0.033862,0.062735,-0.222219,0.129082,-0.075026,0.102772,0.047288,0.282739,0.116910,-0.196864,-0.068466,-0.230079,0.000005,-0.075219,0.086746,-0.081459,0.198271,0.131429,0.164833,0.104517,0.061296,-0.225312,0.071521,-0.182611,-0.167435,0.170808,-0.015743,-0.016329,0.002535,0.039329,...,-0.027212,0.175481,-0.031569,-0.048201,-0.122848,0.264615,0.055259,-0.122896,-0.073748,0.025588,-0.026207,0.110434,-0.153051,0.239316,0.075326,0.244874,0.320699,0.046222,-0.022302,-0.087042,-0.110124,0.065037,-0.062415,0.060719,-8.97873,0.044415,0.048184,-0.005165,-0.039495,0.020332,-0.104020,-0.159283,0.081489,-0.094288,0.081652,-0.091267,0.003904,-0.030987,0.065567,0.015198
2,0.204430,-0.299815,0.278582,-0.330959,0.283490,0.167636,-0.000396,0.186616,-0.080640,-0.002074,0.083603,0.029153,-0.071468,-0.101955,-0.088183,0.027302,-0.042051,0.134516,0.101671,-0.028426,0.077124,-0.186997,-0.313275,-0.250894,0.080097,-0.078265,0.297395,0.140284,-0.008929,0.023978,0.057998,-0.050854,0.088528,0.067004,-0.133049,0.062508,-0.102993,-0.093009,-0.085430,0.076165,...,0.013532,0.145643,0.047052,-0.193214,-0.099032,0.441819,-0.026864,0.064393,-0.085460,-0.063976,0.001207,0.048360,-0.295799,0.250361,0.007193,0.002607,0.367675,0.064131,-0.044522,-0.257064,-0.175361,0.061791,-0.198281,-0.037277,-9.16263,0.137411,0.065680,0.157470,-0.289391,0.087019,-0.099552,-0.125894,0.119627,-0.012324,0.406567,-0.109556,-0.028680,0.201417,-0.064270,0.095350
3,0.074055,-0.994510,0.083315,0.173790,0.435514,0.489149,-0.442962,-0.515874,-0.247308,-0.210775,0.038379,0.199510,-0.283324,-0.483493,-0.898885,0.334158,0.353439,-1.088240,-1.086190,0.170191,-0.323830,-0.223867,-0.157821,-0.040704,-0.135643,0.109425,-0.167469,-0.357555,-0.247309,-0.306711,0.350606,-0.533339,0.937067,0.810881,-0.742399,-0.461994,-0.695902,-0.148049,-0.560312,-0.004402,...,-0.485970,-0.319889,0.264213,-0.837947,0.662875,0.054413,0.096752,-0.052930,-0.357925,0.054988,-0.023470,-0.015924,-0.816854,0.350448,0.005052,-0.324144,0.634460,-0.995354,-0.489603,0.468945,0.522310,1.203780,-0.918136,-0.362671,-1.22464,0.824677,0.418465,0.032358,-0.213618,-0.244904,0.089304,0.333229,0.727644,-0.946214,0.688935,-0.036943,-0.363213,0.905584,1.010070,0.210940
4,0.254428,-0.011411,0.334044,-0.152447,0.132704,-0.040200,0.080763,0.169764,-0.161985,-0.002933,0.075230,0.053451,-0.158602,0.013316,-0.063966,0.150923,0.022072,0.168311,0.124978,-0.229316,0.006148,-0.104214,-0.089166,-0.112851,0.157767,-0.022986,0.275391,0.152245,-0.022349,0.043627,0.068497,-0.072003,0.164927,0.128153,-0.097877,0.112834,0.025989,-0.035945,0.029443,0.111205,...,-0.048833,0.092955,0.000276,-0.139243,-0.096447,0.371564,0.056716,-0.018409,0.003533,-0.065163,0.013406,0.051614,-0.313355,0.178504,-0.017465,0.077003,0.257123,0.087590,-0.035367,-0.082249,-0.172318,0.057436,-0.017893,-0.038150,-9.09304,0.102770,0.073625,0.083781,-0.100582,0.020675,-0

In [14]:
text_embeddings = reddit_df['text_embeddings'].apply(pd.Series)

In [15]:
embeddings = pd.concat([title_embeddings, text_embeddings], axis =1)

In [16]:
embeddings.columns = [i for i in range(1536)]

In [9]:
from category_encoders import OneHotEncoder

In [10]:
subreddit_ce = OneHotEncoder()

In [46]:
subreddits = subreddit_ce.fit_transform(reddit_df['subreddit'])

In [18]:
df = pd.concat([reddit_df['up_votes'],reddit_df['subreddit'], embeddings], axis=1)

In [19]:
df.head()

,up_votes,subreddit,0,1,2,3,4,5,6,7,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,221854,r/AskReddit,0.182144,-0.052698,0.296982,-0.046854,0.066862,-0.110474,0.109145,0.115977,...,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
1,197524,r/AskReddit,0.090396,-0.063034,0.183588,-0.132425,0.218097,-0.061284,0.194140,0.099648,...,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
2,186368,r/AskReddit,0.204430,-0.299815,0.278582,-0.330959,0.283490,0.167636,-0.000396,0.186616,...,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
3,175339,r/AskReddit,0.074055,-0.994510,0.083315,0.173790,0.435514,0.489149,-0.442962,-0.515874,...,-0.120428,-0.059130,0.952337,-1.308660,0.952924,0.261201,-0.729069,0.909033,0.791063,0.122177
4,160311,r/AskReddit,0.254428,-0.011411,0.334044,-0.152447,0.132704,-0.040200,0.080763,0.169764,...,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207


In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('up_votes', axis=1), df['up_votes'])

In [26]:
train_set = (X_train, y_train)

In [31]:
from xgboost import XGBRegressor

In [29]:
model = sk_model()

In [52]:
model.fit(*(train_set))

[01:44:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [58]:
model.score(y_test,pred)

ValueError: ignored

In [57]:
pred = model.predict(X_test)

In [59]:
X_train.head()

,subreddit_1,subreddit_2,subreddit_3,subreddit_4,subreddit_5,subreddit_6,subreddit_7,subreddit_8,subreddit_9,subreddit_10,subreddit_11,subreddit_12,subreddit_13,subreddit_14,subreddit_15,subreddit_16,subreddit_17,subreddit_18,subreddit_19,subreddit_20,subreddit_21,subreddit_22,subreddit_23,subreddit_24,subreddit_25,subreddit_26,subreddit_27,subreddit_28,subreddit_29,subreddit_30,subreddit_31,subreddit_32,subreddit_33,subreddit_34,subreddit_35,subreddit_36,subreddit_37,subreddit_38,subreddit_39,subreddit_40,...,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
11429,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0.225477,0.091579,0.373201,0.024850,-0.161485,0.085262,0.001845,-0.118379,-0.106382,0.102978,-0.118154,-0.042724,-0.300285,0.531707,-0.036965,0.182377,0.195290,-0.127690,0.076081,-0.044050,-0.069825,0.074869,-0.269732,-0.130279,-8.02170,0.006108,0.107337,0.002257,-0.077550,-0.213090,-0.237284,-0.127032,0.137565,-0.142593,0.308050,-0.009991,-0.002525,0.121720,0.097867,-0.041106
2799,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.273151,-0.059469,0.205945,-0.340266,0.320234,-0.104631,-0.317225,-0.647029,-0.521021,-0.077025,-0.237873,0.028217,-0.691589,0.438793,-0.188345,-0.068932,1.090510,-0.445843,-0.439159,0.140661,0.223976,1.168210,-0.543404,-0.289159,-3.82132,0.443102,0.297242,0.115645,-0.576921,-0.736589,0.111816,-0.010130,0.362283,-0.764631,0.505366,0.325662,-0.763775,0.710965,0.409210,0.220402
1642,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.243730,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
3093,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.179900,-0.073909,0.070310,0.066482,-0.262494,-0.145907,-0.530300,0.088729,-0.280738,0.243182,-0.100594,0.201325,-0.359257,-0.072506,-0.187338,-0.001095,0.710825,0.153688,0.064037,-0.128739,-0.222729,-0.375751,0.067073,-0.116550,-3.25659,-0.118616,0.087087,-0.372002,-0.641340,-0.293612,0.303397,-0.294435,-0.540565,-0.239870,-0.162302,-0.099532,-0.323647,0.183855,0.203667,-0.355926
15278,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.182354,-0.352118,0.436717,-0.153194,0.241390,-0.388667,-0.542244,0.001208,0.255917,-0.177232,0.223216,0.131535,-0.424447,0.177258,-0.072651,0.265313,0.435464,-0.003332,0.015523,0.138728,0.390841,-0.177971,-0.496885,-0.177800,-1.24036,0.287297,-0.014849,-0.680968,-0.742825,-1.095090,0.260193,-0.075919,-0.452468,-0.761588,0.283345,0.335066,-0.463583,0.238207,0.085481,-0.230454


In [60]:
X_test.head()

,subreddit_1,subreddit_2,subreddit_3,subreddit_4,subreddit_5,subreddit_6,subreddit_7,subreddit_8,subreddit_9,subreddit_10,subreddit_11,subreddit_12,subreddit_13,subreddit_14,subreddit_15,subreddit_16,subreddit_17,subreddit_18,subreddit_19,subreddit_20,subreddit_21,subreddit_22,subreddit_23,subreddit_24,subreddit_25,subreddit_26,subreddit_27,subreddit_28,subreddit_29,subreddit_30,subreddit_31,subreddit_32,subreddit_33,subreddit_34,subreddit_35,subreddit_36,subreddit_37,subreddit_38,subreddit_39,subreddit_40,...,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
1451,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.243730,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
8793,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0.104173,0.205295,0.322830,-0.145345,-0.323853,0.138185,0.110294,-0.094105,-0.135802,0.018956,-0.094599,-0.091893,-0.425965,0.005144,0.016788,0.341178,0.452654,-0.209478,0.003217,-0.067886,-0.135384,0.363025,-0.292006,0.191587,-7.48267,0.198522,-0.322114,0.032515,-0.277872,-0.156811,-0.091525,0.012621,0.149762,-0.261669,0.454063,0.080677,-0.370884,0.320492,0.259246,-0.321208
12814,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.243730,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
18106,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-0.008551,-0.053723,0.268799,-0.177113,-0.126211,0.411242,-0.155015,-0.250619,-0.122082,-0.137647,0.078025,0.210796,-0.349288,0.260548,-0.167950,0.214914,0.524902,-0.239258,0.073209,-0.099148,-0.344925,0.245079,-0.206464,-0.106118,-8.02177,-0.063310,0.144854,0.157686,-0.373674,-0.336626,-0.136701,-0.087974,0.153318,-0.347229,0.228383,-0.016742,-0.071017,0.166031,0.178876,-0.002682
12720,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,-0.204922,0.201669,0.095816,0.088479,-0.163301,0.178585,0.022706,0.072588,-0.085228,0.241856,-0.052063,0.206275,-0.226309,0.055761,-0.013927,-0.047932,0.055732,0.013788,0.058288,0.074495,-0.144048,0.243730,-0.124181,-0.023625,-8.46837,-0.155591,0.134095,0.142500,0.122580,0.353983,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207


In [61]:
model.score(X_test, y_test)

0.7230084956060623

In [62]:
from sklearn.metrics import mean_absolute_error

In [63]:
mean_absolute_error(pred, y_test)

6877.239513831657

In [64]:
import pickle

In [67]:
with open("up_vote_predicter.pickle", "wb") as f:
    pickle.dump(model, f)

In [12]:
from sklearn.pipeline import make_pipeline

In [30]:
enc = OneHotEncoder()
enc.fit_transform(X_train)

,subreddit_1,subreddit_2,subreddit_3,subreddit_4,subreddit_5,subreddit_6,subreddit_7,subreddit_8,subreddit_9,subreddit_10,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
5444,1,0,0,0,0,0,0,0,0,0,...,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
9643,0,1,0,0,0,0,0,0,0,0,...,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
12720,0,0,1,0,0,0,0,0,0,0,...,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
2059,0,0,0,1,0,0,0,0,0,0,...,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
13125,0,0,0,0,1,0,0,0,0,0,...,-0.250635,-0.079167,0.579189,-1.117600,0.529818,-0.080692,-0.199609,0.943206,0.533414,-0.018670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10213,0,1,0,0,0,0,0,0,0,0,...,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
211,0,0,0,0,0,0,0,0,0,0,...,0.017859,-0.048209,-0.140820,-0.028225,0.431787,-0.342401,0.033433,0.000455,0.178056,0.036207
15953,0,0,0,0,0,0,0,0,0,0,...,-0.123563,-0.029524,0.030757,-0.314385,0.380583,0.336724,0.068898,0.377340,0.013820,0.234301
16011,0,0,0,0,0,0,0,0,0,1,...,-0.101914,0.082471,-0.075933,0.077133,0.193770,-0.203548,0.052253,0.018732,0.116457,-0.027252


In [35]:
pipeline = make_pipeline(OneHotEncoder(), XGBRegressor())

In [36]:
pipeline.fit(*train_set)

Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['subreddit'], drop_invariant=False,
                               handle_missing='value', handle_unknown='value',
                               return_df=True, use_cat_names=False,
                               verbose=0)),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=0, num_parallel_tree=1,
                              objective='re

In [34]:
pipeline

Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['subreddit'], drop_invariant=False,
                               handle_missing='value', handle_unknown='value',
                               return_df=True, use_cat_names=False,
                               verbose=0)),
                ('type', <class 'xgboost.sklearn.XGBRegressor'>)],
         verbose=False)

In [37]:
pipeline.score(X_test, y_test)

0.7087282762411543

In [38]:
from sklearn.metrics import mean_absolute_error

In [39]:
mean_absolute_error(y_test, pipeline.predict(X_test))

6939.68660248509

In [40]:
import pickle

In [56]:
with open("model.pickle", "wb") as f:
    pickle.dump(pipeline, f, protocol=4)

In [43]:
from sklearn.linear_model import LassoCV

In [52]:
pipeline = make_pipeline(OneHotEncoder(), LassoCV(tol=0.001))

In [53]:
pipeline.fit(*train_set)

Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['subreddit'], drop_invariant=False,
                               handle_missing='value', handle_unknown='value',
                               return_df=True, use_cat_names=False,
                               verbose=0)),
                ('lassocv',
                 LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001,
                         fit_intercept=True, max_iter=1000, n_alphas=100,
                         n_jobs=None, normalize=False, positive=False,
                         precompute='auto', random_state=None,
                         selection='cyclic', tol=0.001, verbose=False))],
         verbose=False)

In [54]:
pipeline.score(X_test, y_test)

0.7376746473730273

In [55]:
mean_absolute_error(y_test, pipeline.predict(X_test))

6491.900799453385